In our last [turtorial](subclassing(LayerVsModuleVsModel).ipynb) we learned the difference betweem how we we can create custom `layer` by creating a class subclassed by 
tf.keras.layers.`Layer`, tf.``Module``, tf.keras.``Model``. There we learned how custom layer created with ``Layer`` can take advantages of keras API. So, here we will learn about them. 

The Layer class: the combination of state (weights) and some computation
One of the central abstractions in Keras is the Layer class. A layer encapsulates both a state (the layer's "weights") and a transformation from inputs to outputs
 (a "call", the layer's forward pass)

In [1]:
import tensorflow as tf 

In [2]:
tensor=tf.random.normal(shape=(1,3,2))

In [14]:
"""add_weights()
Why do we use add_weights() method? Why not simply use tf.random_normal()?  
==> The reason is when we create weights using add_weights method the weights becomes part of the layers. That means we can calculate the gradient descent gradient in the training 
using tape.gradient(loss,model.trainable_weights).

We can also create non trainable weights(not tracked during backpropagation.)
""";
class Linear(tf.keras.layers.Layer):

    def __init__(self,out_units):
        super(Linear,self).__init__()
        self.out_units=out_units

    def build(self,input_shape):
        self.w=self.add_weight(shape=(input_shape[-1],self.out_units),initializer='random_normal',trainable=True)
        """
        Here, the weight variable created using .add_weight() can use all the method that a normal tf.Variable can use. For example, .assign_add() etc. 
        """
       
        self.b=self.add_weight(shape=(input_shape[0],self.out_units),initializer='zeros',trainable=True) 

    def call(self,x):
        
        return tf.matmul(x,self.w)+self.b
    

model_dense=Linear(5)
model_dense(tensor)

print("The weights integrated using add_weights method is \n", model_dense.weights)
print("\nThe weights integrated using add_weights method is \n", model_dense.trainable_weights)


The weights integrated using add_weights method is 
 [<tf.Variable 'linear/Variable:0' shape=(2, 5) dtype=float32, numpy=
array([[-0.02546739, -0.01241049, -0.00837238,  0.02654559, -0.04006542],
       [-0.06405389,  0.00607278, -0.02031103,  0.00740079, -0.05004757]],
      dtype=float32)>, <tf.Variable 'linear/Variable:0' shape=(1, 5) dtype=float32, numpy=array([[0., 0., 0., 0., 0.]], dtype=float32)>]

The weights integrated using add_weights method is 
 [<tf.Variable 'linear/Variable:0' shape=(2, 5) dtype=float32, numpy=
array([[-0.02546739, -0.01241049, -0.00837238,  0.02654559, -0.04006542],
       [-0.06405389,  0.00607278, -0.02031103,  0.00740079, -0.05004757]],
      dtype=float32)>, <tf.Variable 'linear/Variable:0' shape=(1, 5) dtype=float32, numpy=array([[0., 0., 0., 0., 0.]], dtype=float32)>]


In [35]:
## adding non trainable weight


class computesum(tf.keras.layers.Layer):

    def __init__(self):
        super(computesum,self).__init__()

    
    def build(self,input_shape):
        print(input_shape)
        self.total=self.add_weight(shape=input_shape,
        initializer='zeros',
        trainable=False
        )

    def call(self,x):
        print(self.total.shape)
        self.total.assign_add(x)
        return self.total
        
print(computesum()(tf.ones((2,2))))

(2, 2)
(2, 2)
<tf.Variable 'computesum_5/Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>


# Layers are recursively composable
This means we can assign instance of custom created layer(Layer in above) to a new custom layer.  It is a best advise to create those sublayers(either custom layers or existing one) in __init__(), and leave it to first __call__() to trigger the building the wights. 

In [39]:
class FFN(tf.keras.layers.Layer):

    def __init__(self):

        super(FFN,self).__init__()

        self.layer1=Linear(10) ## Customly created
        self.layer2=Linear(20)
        self.layer3=tf.keras.layers.Dense(10) ## keras defined layers

    
    def call(self,x):

        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        return x


ffn_output=FFN()(tensor)

print(ffn_output)

tf.Tensor(
[[[ 3.1300467e-03 -5.5321674e-03 -1.1341706e-02  1.6549388e-02
   -1.2665552e-02 -7.5827362e-03  9.3078651e-03 -2.4541318e-03
    8.4767072e-04 -3.1936909e-03]
  [-7.5282576e-04 -2.2420951e-03 -7.8037851e-03  1.1433904e-02
   -6.2538879e-03 -4.7059073e-03  5.4525337e-03 -3.0799480e-03
    1.7661857e-04 -1.3994172e-03]
  [-6.8967789e-04 -9.7735575e-04 -3.9852289e-03  5.8439542e-03
   -2.9316582e-03 -2.3464679e-03  2.6824849e-03 -1.7213271e-03
    4.9050548e-05 -6.2887964e-04]]], shape=(1, 3, 10), dtype=float32)


## add_loss()

In [10]:
"""
add_loss(): This method allows you to add custom losses that are specific to the layer or model you are defining.The added losses can be used for various purposes, such as regularization, 
auxiliary objectives, or other custom objectives that are not directly related to the main output of the model. add_loss() provides flexibility in adding and managing custom losses within the 
layer or model. These losses are typically accessed and included during training through the model.losses attribute, which collects all the losses added using add_loss().

Then, what are some of the benefits of using add_loss() inside the custom layers: 
- It makes it easier to manage multiple losses. If you have multiple losses, you can simply add them all to the model using the add_loss() function. This makes it easier to track and manage the different losses.

- It allows you to customize the way that losses are weighted. When you add a loss to a model using the add_loss() function, you can specify the weight of the loss. This allows you to customize the way that the loss 
is weighted during training.

- It allows you to add losses that are not supported by the tf.keras.Regularization_loss function. The tf.keras.Regularization_loss function only supports L1 and L2 regularization losses.
 However, you can use the add_loss() function to add any custom loss to a model.

 We will see how we can use add_loss() during training in another jupyter notebook files. 
""";
class custom_dense(tf.keras.layers.Layer):

    def __init__(self,out_units,alpha=0.01):
        super(custom_dense,self).__init__()
        self.out_units=out_units
        self.alpha=alpha

    def build(self,input_shape):
        self.w=self.add_weight(shape=(input_shape[-1],self.out_units),initializer='random_normal',trainable=True)
       =self.add_weight(shape=(input_shape[0],self.out_units),initializer='zeros',trainable=True) 

    def call(self,x):
         
        regularization_loss = tf.reduce_sum(tf.square(self.w)) * self.alpha
        self.add_loss(regularization_loss)
        return tf.matmul(x,self.w)+self.b
    

model_dense=custom_dense(5)
model_dense(tensor)

print('model.losses : ', model_dense.losses)

model.losses :  [<tf.Tensor: shape=(), dtype=float32, numpy=0.00024027287>]


In [89]:
"""
get_config() : The get_config() method in a custom tf.keras.layers.Layer subclass is used to return a dictionary that represents the configuration of the layer. 
This configuration dictionary can be used to save and restore the layer, or to create a new layer with the same configuration.

The get_config() method should return a dictionary that contains the following keys:
name: The name of the layer.
**kwargs: Any keyword arguments that were passed to the layer constructor


Why get_config() method is useful? 

- Saving and restoring layers: The get_config() method can be used to save and restore layers. When a layer is saved, the get_config() method is called to get the configuration of the layer. 
The configuration is then stored in the Keras model file. When the model is restored, the get_config() method is called to create a new layer with the same configuration.

- Creating new layers with the same configuration: The get_config() method can be used to create new layers with the same configuration. For example, you can use the get_config() method to 
create a new layer that has the same configuration as a layer that you have already trained.

- Serializing and deserializing layers: The get_config() method can be used to serialize and deserialize layers. Serialization is the process of converting a layer to a format that can be stored or transmitted. 
Deserialization is the process of converting a layer from a stored or transmitted format to a layer object.
""";

class custom_dense(tf.keras.layers.Layer):

    def __init__(self,out_units,alpha=0.01,name='test_model'):
        super(custom_dense,self).__init__(name=name)
        self.out_units=out_units
        self.alpha=alpha

    def build(self,input_shape):
        self.w=self.add_weight(shape=(input_shape[-1],self.out_units),initializer='random_normal',trainable=True)
        self.b=self.add_weight(shape=(input_shape[0],self.out_units),initializer='zeros',trainable=True) 

    
    def get_config(self):

        config={
            'name': self.name,
            'alpha': self.alpha,
            'units': self.out_units
        }

        return config



    def call(self,x):
        
        regularization_loss = tf.reduce_sum(tf.square(self.w)) * self.alpha
        self.add_loss(regularization_loss)
        return tf.matmul(x,self.w)+self.b
    
    
model_dense=custom_dense(5)
model_dense(tensor)



<tf.Tensor: shape=(1, 3, 5), dtype=float32, numpy=
array([[[-0.0090224 , -0.00258729, -0.01169324,  0.01552782,
         -0.00435234],
        [ 0.04467518,  0.05567386, -0.1724642 ,  0.05668667,
         -0.08487829],
        [ 0.01611871, -0.00021536,  0.04688975, -0.0428163 ,
          0.01978744]]], dtype=float32)>

Now we will create a new_model seqential model with above defined subclass custom layers. And we will know how we can use these configs. 

In [92]:
inputs=tf.keras.Input(shape=(3,2))
x=model_dense(inputs)
outputs=tf.keras.layers.Dense(10)(x)
model=tf.keras.Model(inputs,outputs,name='okokok')


In [95]:
## Try tomorrow 

In [94]:
model.save('model_with_config/')

AttributeError: 'NoneType' object has no attribute 'replace'

In [61]:
model = tf.keras.models.Sequential([
    custom_dense(100, alpha=0.01),
    tf.keras.layers.Dense(10),
])

model.save("model.h5")

# Restore the model
model = tf.keras.models.load_model("model.h5")

# Print the configuration of the custom_dense layer
print(model.layers[0].get_config())


ValueError: Weights for model sequential_4 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.